In [ ]:
import pandas as pd
import os

In [ ]:
path = "path to fills.csv"
fillsDF = pd.read_csv(path)

fillsDF.head(15)

In [ ]:
# NOTE: I have lost the code that scraped the data for the object "shit" but it was generated by scraping the transactions history aggregated from CryptoTrader.Tax
# It would be wise to write a custom transaction history aggregator using APIs from various platforms but for portability to other accounts but merely having the data saved should be good enough for just me for now.

formatted_fills = [','.join(shit[i-3:i]).replace("\t",",").replace("\n","").replace(',,',',').replace('Coinbase-Pro/GDAX','Coinbase Pro') for i in range(3,len(shit),3)]

formatted_fills_df = pd.DataFrame(formatted_fills)
formatted_fills_df[['Timestamp', 'Exchange', 'Type', 'Product', 'Traded Amount', 'Traded Currency', 'Received Amount', 'Received Currency']] = formatted_fills_df[0].str.split(',',expand=True).drop(columns=8)

formatted_fills_df = formatted_fills_df.drop(columns=0)

formatted_fills_df.to_csv("Fills_start-2019.csv",index=False)

In [ ]:
formatted_fills_df= pd.read_csv("Fills_start-2019.csv")
formatted_fills_df.head(30)

In [ ]:
currencies = list(formatted_fills_df['Received Currency'].unique())

In [ ]:
# Import Dependencies
import dateutil.parser as parser
import cbpro
import datetime
import time
import progressbar

# Initial declarations
public_client = cbpro.PublicClient() # Establish client connection for Coinbase Pro API. Used for historical price lookups.
formatted_fills_df= pd.read_csv("Fills_start-2019.csv") # Read in CSV with fills data.
bar = progressbar.ProgressBar(max_value=len(formatted_fills_df)) # Create Progress bar.
Transactions = [] # Will contain each cost basis/proceeds analysis.
k={} # Defines where to start looking for the first buy for each sell.

for i in formatted_fills_df.index: # For each row in CSV:
    row = formatted_fills_df.iloc[i]
    if row['Type'] == 'SELL': # Find the next available row with a SELL status.
        Sell_Date = row['Timestamp']
        Sell_Traded_Currency = row['Traded Currency']
        Sell_Traded_Amount = row['Traded Amount']
        Sell_Received_Currency = row['Received Currency']
        Sell_Received_Amount = row['Received Amount'] #Proceeds
        Sale_Exchange = row['Exchange']
        # Converting the sale currency to USD using historical fair market price if it is not already.
        if Sell_Received_Currency == 'USD':
            Sell_Received_Amount_USD = Sell_Received_Amount
        else:
            start = parser.parse(Sell_Date)
            end = start + datetime.timedelta(minutes = 1)
            currency_pair = f'{Sell_Received_Currency}-USD'
            Spot_Price = public_client.get_product_historic_rates(product_id=currency_pair, start=start, end=end)[0][2]
            Sell_Received_Amount_USD = Sell_Received_Amount * Spot_Price
            time.sleep(1)

        Buy_Dates = []
        Asset_Amounts = []
        Buy_Traded_Currencies = []
        Buy_Traded_Amounts = []
        Buy_Prices_USD = []
        Buy_Traded_Amounts_USD = []
        Buy_Received_Amounts = []
        Buy_Received_Currencies = []
        Buy_Lines = []
        Purchase_Exchanges = []
        
        if Sell_Traded_Currency not in k:
            k.update({Sell_Traded_Currency: 0})
        
        for n in range(k[Sell_Traded_Currency], i+1):
            nrow = formatted_fills_df.iloc[n]
            Buy_Date = nrow['Timestamp']
            Buy_Traded_Currency = nrow['Traded Currency']
            Buy_Traded_Amount = nrow['Traded Amount']
            if Buy_Traded_Currency == 'USD':
                Buy_Traded_Amount_USD = Buy_Traded_Amount
                Spot_Price = Buy_Traded_Amount / Buy_Received_Amount
            else:
                start = parser.parse(Buy_Date)
                end = start + datetime.timedelta(minutes = 1)
                currency_pair = f'{Buy_Traded_Currency}-USD'
                Spot_Price = public_client.get_product_historic_rates(product_id=currency_pair, start=start, end=end)[0][2]
                Buy_Traded_Amount_USD = nrow['Traded Amount'] * Spot_Price
                time.sleep(1)
                
            Buy_Received_Currency = nrow['Received Currency']
            Buy_Received_Amount = nrow['Received Amount']
            Purchase_Exchange = nrow['Exchange']
            if Buy_Received_Currency == Sell_Traded_Currency:
                Buy_Lines.append(n)
                
                if Sell_Traded_Amount > sum(Buy_Received_Amounts) + Buy_Received_Amount:
                    
                    Buy_Received_Amounts.append(Buy_Received_Amount)
                    Buy_Traded_Amounts.append(Buy_Traded_Amount)
                    Buy_Traded_Amounts_USD.append(Buy_Traded_Amount_USD)
                    Buy_Prices_USD.append(Spot_Price)
                    Buy_Traded_Currencies.append(Buy_Traded_Currency)
                    Buy_Dates.append(str(Buy_Date))
                    Buy_Received_Currencies.append(Buy_Received_Currency)
                    Purchase_Exchanges.append(Purchase_Exchange)
                    
                elif Sell_Traded_Amount == sum(Buy_Received_Amounts) + Buy_Received_Amount:
                    Buy_Received_Amounts.append(Buy_Received_Amount)
                    Buy_Traded_Amounts.append(Buy_Traded_Amount)
                    Buy_Traded_Amounts_USD.append(Buy_Traded_Amount_USD)
                    Buy_Prices_USD.append(Spot_Price)
                    Buy_Traded_Currencies.append(Buy_Traded_Currency)
                    Buy_Dates.append(str(Buy_Date))
                    Buy_Received_Currencies.append(Buy_Received_Currency)
                    Purchase_Exchanges.append(Purchase_Exchange)
                    k[Sell_Traded_Currency] = n
                    break
                elif Sell_Traded_Amount < sum(Buy_Received_Amounts) + Buy_Received_Amount:
                    Adjusted_Buy_Received_Amount = Sell_Traded_Amount - sum(Buy_Received_Amounts)
                    Adjusted_Buy_Traded_Amount = Buy_Traded_Amount * Adjusted_Buy_Received_Amount / Buy_Received_Amount
                    Adjusted_Buy_Traded_Amount_USD = Buy_Traded_Amount_USD * Adjusted_Buy_Received_Amount / Buy_Received_Amount
                    
                    Buy_Received_Amounts.append(Adjusted_Buy_Received_Amount)
                    Buy_Traded_Amounts.append(Adjusted_Buy_Traded_Amount)
                    Buy_Traded_Amounts_USD.append(Adjusted_Buy_Traded_Amount_USD)
                    Buy_Prices_USD.append(Spot_Price)
                    Buy_Traded_Currencies.append(Buy_Traded_Currency)
                    Buy_Dates.append(str(Buy_Date))
                    Buy_Received_Currencies.append(Buy_Received_Currency)
                    Purchase_Exchanges.append(Purchase_Exchange)
                    
                    New_Received_Amount = Buy_Received_Amount - Adjusted_Buy_Received_Amount
                    New_Traded_Amount = Buy_Traded_Amount - Adjusted_Buy_Traded_Amount
                    formatted_fills_df.at[n,'Received Amount'] = New_Received_Amount
                    formatted_fills_df.at[n,'Traded Amount'] = New_Traded_Amount
                    k[Sell_Traded_Currency] = n
                    break
            if n == i:
                UntraceableFunds = True
                break
            else:
                UntraceableFunds = False
        Cost_Basis = sum(Buy_Traded_Amounts_USD)
        Gains = Sell_Received_Amount_USD - Cost_Basis
        Transaction = {'Purchase Dates': Buy_Dates,
                       'Purchase Exchanges': Purchase_Exchanges,
                       'Asset Purchased': Buy_Received_Currencies,
                       'Asset Amount': Buy_Received_Amounts,
                       'Purchase Amounts Total': sum(Buy_Received_Amounts),
                       'Currencies Traded for Asset': Buy_Traded_Currencies,
                       'Amounts Traded': Buy_Traded_Amounts,
                       'Amounts Traded USD value': Buy_Traded_Amounts_USD,
                       'Sell Date': Sell_Date,
                       'Sale Exchange': Sale_Exchange,
                       'Asset Sold for': Sell_Received_Currency,
                       'Asset Amount Sold': Sell_Traded_Amount,
                       'Cost Basis': Cost_Basis,
                       'Proceeds': Sell_Received_Amount_USD,
                       'Gains': Gains}
        Transactions.append(Transaction)
        bar.update(i+1)
        if UntraceableFunds == True:
            print(f'ERROR: Insufficient funds were found to have performed the sale on line {i + 1}. Please ensure the validity of the data in your input CSV and try again. It must contain all buys and received currency to be accurate across exchanges in order to be accurate. The buys that were attempted to be used to explain this sale were on lines{Buy_Lines}.Line {i + 1}:')
            print(row)
            break

FIFO_Tax_DF = pd.DataFrame(Transactions)
start = formatted_fills_df.iloc[0]['Timestamp'].split()[0].replace('/','-')
end = formatted_fills_df.iloc[-1]['Timestamp'].split()[0].replace('/','-')
filename = f'FIFO_Gains_Tax_Analysis_{start}_to_{end}.csv'
FIFO_Tax_DF.to_csv(filename, index=False)

print(f'Total Gains all time: {FIFO_Tax_DF["Gains"].sum()}')

DF_2019 = FIFO_Tax_DF[FIFO_Tax_DF['Sell Date'].str.contains('2019')].reset_index(drop=True)

print(f'Total Gains for 2019: {DF_2019["Gains"].sum()}')

In [ ]:
Analyses = []
for i in DF_2019.index:
    row = DF_2019.iloc[i]
    # Purchase Lists
    Purchase_Dates = row['Purchase Dates']
    Purchase_Exchanges = row['Purchase Exchanges']
    Asset_Purchased = row['Asset Purchased']
    Asset_Amounts = row['Asset Amount']
    Currencies_Traded_for_Asset = row['Currencies Traded for Asset']
    Amounts_Traded = row['Amounts Traded']
    Amounts_Traded_USD_value = row['Amounts Traded USD value']
    
    for i in range(0,len(Purchase_Dates)):
        
        Asset_Amount = Asset_Amounts[i]
        
        # Sale data
        Sell_Date = row["Sell Date"].split()[0]
        Asset_Sold_for = row["Asset Sold for"]
        Asset_Amount_Sold = row["Asset Amount Sold"]
        Sale_Exchange = row['Sale Exchange']
        
        # Analysis data
        Cost_Basis = Amounts_Traded_USD_value[i]
        Preceeds = row["Proceeds"]
        Proceeds = float(Preceeds) * float(Asset_Amount) / float(Asset_Amount_Sold)
        Gains = Proceeds - Cost_Basis
        
        Analysis = {'Purchase_Date': Purchase_Dates[i].split()[0],
                    'Purchase_Exchanges': Purchase_Exchanges[i],
                    'Asset_Purchased': Asset_Purchased[i],
                    'Asset_Amount': Asset_Amount,
                    'Currencies_Traded_for_Asset': Currencies_Traded_for_Asset[i],
                    'Amounts_Traded': Amounts_Traded[i],
                    'Cost_Basis': Cost_Basis,
                    'Sell_Date': Sell_Date,
                    'Sale_Exchange': Sale_Exchange,
                    'Asset_Sold_for': Asset_Sold_for,
                    'Asset_Amount_Sold': Asset_Amount_Sold,
                    'Proceeds': row["Proceeds"] * Asset_Amount / Asset_Amount_Sold,
                    'Gains': Gains}
        Analyses.append(Analysis)
FIFO_Buys_Breakdown = pd.DataFrame(Analyses)

start = FIFO_Buys_Breakdown.iloc[0]['Purchase_Date'].replace('/','-')
end = FIFO_Buys_Breakdown.iloc[-1]['Sell_Date'].replace('/','-')
filename = f'FIFO_Gains_Tax_Analysis_Buys_Breakdown_{start}_to_{end}.csv'
FIFO_Buys_Breakdown.to_csv(filename, index=False)

In [ ]:
# filename = f'FIFO_Gains_Tax_Analysis_Buys_Breakdown_11-12-2017_to_5-10-2019.csv'
# FIFO_Buys_Breakdown = pd.read_csv(filename)
TurboTaxFormat = FIFO_Buys_Breakdown[['Sale_Exchange','Asset_Purchased','Purchase_Date','Cost_Basis','Sell_Date','Proceeds','Gains']]
TurboTaxFormat.to_csv('TTF_2019_uncompressed.csv', index=False)

In [ ]:
TTF = TurboTaxFormat
TTF

In [ ]:
TTF = TurboTaxFormat
analyses = []
for i in TTF.index:
    row = TTF.iloc[i]
    rows = TTF[TTF['Asset_Purchased'].str.contains(row['Asset_Purchased'])][TTF['Purchase_Date'].str.contains(row['Purchase_Date'])][TTF['Sell_Date'].str.contains(row['Sell_Date'])]
    analysis = {'Sale Exchange': rows.iloc[-1]['Sale_Exchange'],
                'Asset Purchased': row['Asset_Purchased'],
                'Purchase Date': row['Purchase_Date'],
                'Cost Basis': rows['Cost_Basis'].sum(),
                'Sell Date': row['Sell_Date'],
                'Proceeds': rows['Proceeds'].sum(),
                'Gains': rows['Gains'].sum()}
    analyses.append(analysis)
analDF = pd.DataFrame(analyses)
analDF = analDF.drop_duplicates()
analDF.to_csv('TTF_2019_compressed.csv',index=False)

In [ ]:
analDF = analDF.reset_index(drop=True)
analDF